In [ ]:
import re
from collections import defaultdict
from pathlib import Path

from tqdm.notebook import tqdm
from itertools import chain

import githelpers
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [ ]:
cached = ghapicache.GhApiCache()
ghapi = cached.api

In [ ]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

In [ ]:
all_tags = githelpers.git("tag", "--list")

# Filter tags matching the version pattern v(a.b.c) and extract the decimal component
#re_version = re.compile(r"^v(\d+\.\d+\.\d+)$")
re_version = re.compile(r"^v(\d+\.\d+\.\d+)$")
version_tuples = [tuple(int(v) for v in m.group(1).split('.')) for tag in all_tags
                   if (m := re_version.match(tag))]
#versions = [m.group(1) for tag in all_tags if (m := re_version.match(tag))]
# print(versions)

grouped_versions = defaultdict(list)
for major, minor, patch in version_tuples:
    grouped_versions[(major, minor)].append(patch)
grouped_versions.pop((0, 0), None)  # Remove the initial version (0.0.0)
# Find the last patch release for each minor release
last_version = {key: max(patches) for key, patches in grouped_versions.items()}

# Zenodo release

- All versions should list contributors since the initial last major release.
- Team members should be edited at the first release, then they'll be copied later.

In [ ]:
def new_series(major):
    patch = 0
    release = f"0.{major}"
    return release_notes.ReleaseMetadata(
        release=release,
        merge_bases=[f"v0.{major - 1}.0"],
        target_branch=f"v{release}.{patch}",
    )


def new_version(major, patch):
    release = f"0.{major}.{patch}"
    prev_patch = last_version.get((0, major - 1), 0)
    merge_bases = [f"v0.{major - 1}.{prev_patch}"]
    if patch != 0:
        merge_bases = [f"v0.{major}.{patch - 1}"] + merge_bases

    return release_notes.ReleaseMetadata(
        release=release, merge_bases=merge_bases, target_branch="v" + release
    )


def comprehensive_version(major, patch):
    "This includes all authors since the previous release split off"
    release = f"0.{major}.{patch}"
    merge_bases = [f"v0.{major}.0-dev"]

    return release_notes.ReleaseMetadata(
        release=release, merge_bases=merge_bases, target_branch="v" + release
    )

def since_last_release(major, patch):
    if patch == 0:
        prev_patch = last_version[(0, major - 1)]
        prev = f"0.{major - 1}.{prev_patch}"
    else:
        prev = f"0.{major}.{patch - 1}"
    release = f"0.{major}.{patch}"
    return release_notes.ReleaseMetadata(
        release=release, merge_bases=[f"v{prev}"], target_branch=f"v{release}"
    )

In [ ]:
print(new_series(1))
print(new_series(5))
print(new_version(5,1))
print(new_version(5,2))
print(comprehensive_version(5,2))

In [ ]:
def load_contributions(release_md):
    prs = release_notes.PullRequestRange(release_md)
    count_contrib = release_notes.ContributionCounter(cached)
    for pr in tqdm(prs.pull_ids):
        count_contrib(pr)

    # Create author list
    return count_contrib.sorted()

make_zenodo_md = release_notes.ZenodoMetadataBuilder(user_cache=user_cache, teams=TEAMS)

In [ ]:
import zenodoapi
%aimport zenodoapi

In [ ]:
# Load the Zenodo token
#token_path = Path.home() / ".config/zenodo-sandbox-token"
token_path = Path.home() / ".config/zenodo-token"
with open(token_path) as f:
    zenodo_token = f.read().strip()

zenodo = zenodoapi.Zenodo(zenodo_token)
zenodo.api_url = "https://zenodo.org/api/"
#assert "sandbox" in zenodo.api_url

## Upload major versions

Major and minor versions should all look the same 

In [ ]:
grouped_versions

In [ ]:
def download_tarball(gh_release):
    asset = gh_release['assets'][0]
    name = asset['name']
    content = cached.download_file(asset['url'], ext=Path(name).suffix)
    assert content is not None
    return (content, name)

gh_release = release_notes.find_release(ghapi, "0.2.0")
(content, name) = download_tarball(gh_release)

In [ ]:
urls = []
depositions = {}
for major, minor in tqdm(sorted(grouped_versions.keys())):
    assert major == 0
    release_md = comprehensive_version(minor, 0)
    title = f"Celeritas {major}.{minor}"
    deposition = zenodo.find_deposition(title)
    if deposition is None:
        gh_release = release_notes.find_release(ghapi, release_md.release)
        new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
        new_md["title"] = title

        # Create the Zenodo deposition
        deposition = zenodo.create_deposition(new_md)
    if not deposition.get_files():
        # Upload the release tarball
        (content, name) = download_tarball(gh_release)
        deposition.upload(content, name)
    depositions[(major, minor)] = deposition
    urls.append(deposition.html)

print("\n".join(urls))
print("""
IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses
""")

## Upload minor version

In [ ]:
def create_minor_version(dep, minor, patch):
    release_md = comprehensive_version(minor, patch)
    gh_release = release_notes.find_release(ghapi, release_md.release)

    # Get the Zenodo metadata    
    old_md = dep.data["metadata"]
    new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
    # Only update editors, not team members
    new_contrib = [u for u in new_md["contributors"] if u["type"] == "Editor"]
    old_contrib = [u for u in dep.md["contributors"] if u["type"] != "Editor"]
    new_md["contributors"] = new_contrib + old_contrib
    # Don't change the title
    new_md["title"] = dep.data["metadata"]["title"]
    # Add the body
    new_md["description"] = "\n\n".join(
        [
            old_md["description"],
            f"<h1>Version {release_md.release}</h1>",
            new_md["description"],
        ]
    )

    new_vers = dep.create_new_version()
    new_vers.update(new_md)
    # Upload the release
    (content, name) = download_tarball(gh_release)
    new_vers.upload(content, name)
    # The old tarball may still be there (this is buggy) so delete it
    new_vers.refresh()
    for file in new_vers.get_files():
        if file.filename != name:
            try:
                file.delete()
            except Exception as e:
                print(f"Failed to delete {file.filename}: {e}")
    new_vers.refresh()
    return new_vers

In [ ]:
depositions

In [ ]:
# Create the next version update for each branch
for (major, minor), patches in grouped_versions.items():
    dep = depositions[(major, minor)].refresh()
    assert patches[0] == 0
    for patch in patches[1:]:
        ver_key = (major, minor, patch)
        try:
            # Access published version
            new_vers = depositions[ver_key]
        except KeyError:
            prevdep = zenodo.get_deposition(dep.get_latest_version().id)
            new_vers = create_minor_version(prevdep, minor, patch)
            depositions[ver_key] = new_vers
            print(f"Created minor version {major}.{minor}.{patch}:", new_vers.html)
            break
        else:
            print(f"Existing minor version {major}.{minor}.{patch}:", new_vers.html)

In [ ]:
cached.flush()